In [1]:
# !pip install langchain deeplake openai tiktoken

In [12]:
import os
import getpass
from langchain.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

# Setting the OPENAI KEY and ACTIVE LOOP Vector DB

os.environ["OPENAI_API_KEY"] = ""
os.environ["ACTIVELOOP_TOKEN"] = ""

In [2]:
# disallowed_special=() is required to avoid Exception: 'utf-8' codec

embeddings = OpenAIEmbeddings(disallowed_special=())

In [5]:
# Cloning a REPO
# git clone https://github.com/nazfisa/CRUD-Operation-Using-SpringMVC-RESTAPI/tree/master

# Load all files inside the repository

root_dir = "./Simple-SpringFramework-Application"
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

In [7]:
# Chunking all files

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [9]:
# Executing Indexing - computes embeddings and uploads to Activeloop 

username = "pratik"  
db = DeepLake(
    dataset_path=f"hub://{username}/simple-springframework-app",
    embedding=embeddings,
)
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!


100%|██████████| 87/87 [00:04<00:00, 18.63it/s]


Dataset(path='hub://pratik/simple-springframework-app', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (87, 1)      str     None   
 metadata     json      (87, 1)      str     None   
 embedding  embedding  (87, 1536)  float32   None   
    id        text      (87, 1)      str     None   


['c8a038fa-6458-11ee-b60b-c03c592e8a14',
 'c8a038fb-6458-11ee-98a7-c03c592e8a14',
 'c8a038fc-6458-11ee-b082-c03c592e8a14',
 'c8a038fd-6458-11ee-b8f9-c03c592e8a14',
 'c8a038fe-6458-11ee-9aea-c03c592e8a14',
 'c8a038ff-6458-11ee-8297-c03c592e8a14',
 'c8a03900-6458-11ee-9092-c03c592e8a14',
 'c8a03901-6458-11ee-bdca-c03c592e8a14',
 'c8a03902-6458-11ee-a0b8-c03c592e8a14',
 'c8a03903-6458-11ee-9f19-c03c592e8a14',
 'c8a03904-6458-11ee-99ea-c03c592e8a14',
 'c8a03905-6458-11ee-bac4-c03c592e8a14',
 'c8a03906-6458-11ee-b229-c03c592e8a14',
 'c8a03907-6458-11ee-b51c-c03c592e8a14',
 'c8a03908-6458-11ee-9369-c03c592e8a14',
 'c8a03909-6458-11ee-bdf0-c03c592e8a14',
 'c8a0390a-6458-11ee-9e80-c03c592e8a14',
 'c8a0390b-6458-11ee-96ea-c03c592e8a14',
 'c8a0390c-6458-11ee-a556-c03c592e8a14',
 'c8a0390d-6458-11ee-b910-c03c592e8a14',
 'c8a0390e-6458-11ee-ba1e-c03c592e8a14',
 'c8a0390f-6458-11ee-8dfa-c03c592e8a14',
 'c8a03910-6458-11ee-a04c-c03c592e8a14',
 'c8a03911-6458-11ee-88e9-c03c592e8a14',
 'c8a03912-6458-

In [25]:
# Loading the dataset

db = DeepLake(
    dataset_path=f"hub://{username}/simple-springframework-app",
    read_only=True,
    embedding=embeddings,
)

Deep Lake Dataset in hub://pratik/simple-springframework-app already exists, loading from the storage


In [26]:
# Constructing the retriever

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [27]:
# Constructing the Conversational Chain

model = ChatOpenAI(model_name="gpt-3.5-turbo")  # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [ ]:
chat_history = []

In [32]:
questions = [
    "What is the SpringBoot version of this SrpingFramework dependency 'spring-webmvc'?"
]

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is the SpringBoot version of this SrpingFramework dependency 'spring-webmvc'? 

**Answer**: The provided POM file does not contain any dependencies related to Spring Boot. Therefore, it is not possible to determine the Spring Boot version of the 'spring-webmvc' dependency based on the given information. 

